In [3]:
from pylab import *
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from sklearn import preprocessing
from sklearn.metrics import pairwise
import itertools


starting file
convert data to lists
converting training data to list
Max movie counted: 65132
converting index and frequency lists to csc and article lists
in jagged to csc method
maxcolumn: 65133
creating sparse csr of movies
rows: 71534 	 cols: 65133
------------finished-------------


In [4]:
max_movies = 65133 # based on actors directors and genres file
def file_to_list(file, verbose=False):
    if verbose:
        print("converting training data to list")
    file = 'data/' + file
    max_movie_count = 0
    movie_list = []
    user_list = []
    rating_list = []
    with open(file, 'r') as csv:
        movie_num = 0
        movie_freqs = [0] * max_movies
        csv.readline()  # skip header
        for row in csv:
            # each row is are movie indexes paired with their counts, all separated by spaces
            row_list = row.split(' ')
            user_id = int(row_list[0]) - 1
            movie_id = int(row_list[1]) - 1
            rating = float(row_list[2])
            if movie_id > max_movie_count:
                max_movie_count = movie_id
            movie_freqs[movie_id] += 1

            # each appended item is a article with its movie (reference indexes) and respective counts
            # creates jagged list with varying length reviews
            user_list.append(user_id)
            movie_list.append(movie_id)
            rating_list.append(rating)
    if verbose:
        print("Max movie counted: {}".format(max_movie_count))

    return user_list, movie_list, rating_list, movie_freqs


In [5]:
def lists_to_csc(user_list, movie_list, rating_list, verbose):
    if verbose:
        print("in jagged to csc method")
    max_movie_index = max(movie_list) + 1
    max_user_index = user_list[len(user_list) - 1] + 1
    if verbose:
        print("maxcolumn: {}".format(max_movie_index))
    coo = create_coo(user_list, movie_list, rating_list, max_user_index, max_movie_index, verbose)
    return csr_matrix(coo)


In [6]:
# helper method creates sparse coordinate format matrix for fast construction
def create_coo(rows, cols, values, num_rows, num_cols, verbose):
    if verbose:
        print('creating sparse csr of movies')
        print("rows: {0} \t cols: {1}".format(num_rows, num_cols))
    # in create csr
    sparse_coo = coo_matrix((values, (rows, cols)), shape=(num_rows, num_cols), dtype=np.float32)
    return sparse_coo


In [7]:
# t = training data, x = test data
def write_csc_to_disk(csc, fn, verbose):
    if verbose:
        print('csc file {0} to disk'.format(fn))
    fn = 'data/' + fn
    np.savez(fn, data=csc.data, indices=csc.indices, indptr=csc.indptr, shape=csc.shape)


In [8]:
def write_list_to_disk(dense_bag_of_movies, fn, verbose):
    if verbose:
        print("writing articles to list {}".format(fn))
    with open(fn, 'w') as csv:
        for row in dense_bag_of_movies:
            csv.write("{0}\n".format(row))


In [9]:
def write_list_to_disk(dense_bag_of_movies, fn, verbose):
    if verbose:
        print("writing articles to list {}".format(fn))
    with open(fn, 'w') as csv:
        for row in dense_bag_of_movies:
            csv.write("{0}\n".format(row))


In [10]:
def get_fns(full_run=True):
    if full_run:
        rating_fn = 'train.dat'
        movie_actors_fn = 'movie_actors.latin1.dat'
        movie_directors_fn = 'movie_directors.latin1.dat'
        movie_genres_fn = 'movie_genres.dat'
        movie_tags_fn = 'movie_tags.dat'
        user_tags_fn = 'user_taggedmovies.dat'
        tags_fn = 'tags.dat'
        test_fn = 'test.dat'
    else:
        short_dir = 'short/'
        rating_fn = short_dir + 'train_short.dat'
        movie_actors_fn = short_dir + 'movie_actors.latin1_short.dat'
        movie_directors_fn = short_dir + 'movie_directors.latin1_short.dat'
        movie_genres_fn = short_dir + 'movie_genres_short.dat'
        movie_tags_fn = short_dir + 'movie_tags_short.dat'
        user_tags_fn = short_dir + 'user_taggedmovies_short.dat'
        tags_fn = short_dir + 'tags_short.dat'
        test_fn = short_dir + 'test_short.dat'
    return rating_fn, movie_actors_fn, movie_directors_fn, movie_genres_fn, movie_tags_fn, user_tags_fn, tags_fn, test_fn


In [14]:
verbose = True
full_run = True
frequency_write = True
if verbose:
    print('starting file')

rating_fn, movie_actors_fn, movie_directors_fn, movie_genres_fn, movie_tags_fn, user_tags_fn, tags_fn, test_fn = \
    get_fns(full_run)
# convert data to lists
if verbose:
    print("convert data to lists")
user_list, movie_list, rating_list, movie_freqs = file_to_list(rating_fn, verbose)

# convert lists to cscs
if verbose:
    print("converting index and frequency lists to csc and article lists")
rating_csc = lists_to_csc(user_list, movie_list, rating_list, verbose)

if verbose:
    print("------------finished-------------")


NameError: name 'rating_csc' is not defined

In [50]:
def delete_csc_rows(csc, indices):
    bool_indx = np.ones(csc.shape[0], dtype=bool)
    len_string = "{} {} {}".format(rating_csc.shape[0], indices.shape[0], bool_indx.shape[0])
    print(len_string)
    return  csc[indices]

In [53]:
sums = rating_csc.sum(axis=1)#row sums
indices = sums > 0
trimmed_csc = delete_csc_rows(rating_csc, indices)
print(trimmed_csc)

71534 71534 71534
[[ 0.  0.  0. ...,  0.  0.  0.]]


In [47]:
test_array = np.array([1,2,3,4,5])
index_array = np.array([True,True, False, True, True])
test_array[index_array]=1
print(test_array)
# print(len(test_array))
# print(len(index_array))

[1 1 3 1 1]
